In [1]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time

import tqdm
import pandas as pd

import re

In [82]:
# 데이터프레임 생성
naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
#options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

#Step 1. 크롬 웹브라우저 실행
#driver = webdriver.Chrome(chrome_path, options=options)

In [83]:
query = "신창풍차"
start_date = "20200301"
end_date = "20211001"

In [102]:
driver.get('http://www.naver.com')
time.sleep(0.5)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query)  # query_txt는 위에서 입력한 '이재용'
element.submit()

#Step 3. "블로그" 카테고리 선택
driver.find_element_by_link_text("VIEW").click( )
driver.find_element_by_link_text("블로그").click( )

time.sleep(1)

#Step 4. 오른쪽의 검색 옵션 버튼 클릭
#**자바 스크립트로 클릭되는 것은 click()이 안먹히니 send_keys 이용하기
op = driver.find_element_by_xpath("""//*[@id="snb"]/div[1]/div/div[2]/a""")
op.send_keys(Keys.ENTER)
time.sleep(1)

#Step 5. 날짜 입력
d = driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]""")
d.send_keys(Keys.ENTER)
time.sleep(1)

# 시작 날짜 입력하기
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[1]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[18]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[3]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)

# 종료 날짜 입력하기
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[10]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)

# Step6. 날짜 입력 후 "적용하기" 버튼 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button""").click()
time.sleep(1)

# 1차 크롤링 (url, 제목) 시작
total_page = 100
for i in range(0, total_page):  # 페이지 번호
    a = driver.find_element_by_css_selector("#sp_blog_{} > div > div > a".format(i+1))
    
    # url 수집
    url = a.get_attribute('href')
    
    # 제목 수집
    title = a.text

    naver_df.loc[i] = ['nan', '네이버', query, title, 'nan', url, 'nan']
    
    # 스크롤 내리기
    if i % 30 == 0:
        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        

In [128]:
# 2차 크롤링 (블로그 제목, 포스팅 날짜, 글쓴이) 시작
for i in range(len(naver_df)):
    driver.get(naver_df.loc[i]['url'])
    driver.switch_to.frame('mainFrame')
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements_by_css_selector(overlays)

    content_list = ""

    for content in contents:
        content_list = content_list + content.text

    # 글쓴이 수집
    author = driver.find_element_by_xpath("""//*[@id="nickNameArea"]""").text
    
    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df.loc[i][['날짜', '내용', '글쓴이']] = [date, content_list, author]

In [132]:
naver_df.tail()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
95,2021. 8. 9. 10:11,네이버,신창풍차,"제주여행6일차 해안드라이브코스 신창풍차해안도로,티뮤지움...",,https://blog.naver.com/bcricket2?Redirect=Log&...,바리스타너구리
96,2021. 8. 6. 7:30,네이버,신창풍차,제주 일몰 명소 신창풍차해안도로 경치좋은 곳 강추!!!,,https://blog.naver.com/silverpixy?Redirect=Log...,픽시케익
97,2021. 8. 5. 11:14,네이버,신창풍차,"""구해줘 홈즈""에 출연했던 신창풍차해안 집입니다.",제주도가 사실상 유명무실해진 부동산 투자이민제도의 성과를 분석하고 개선방안을 마련하...,https://blog.naver.com/miraeroac?Redirect=Log&...,미래로
98,2021. 5. 30. 16:51,네이버,신창풍차,신창풍차해안도로 마실,오늘은 오랜만에 신창풍차해안도로 마실 나옴\n어제 우도 달려놓고 이렇게까지 놀고 싶...,https://blog.naver.com/nobokim?Redirect=Log&lo...,Dohee
99,2021. 9. 30. 7:22,네이버,신창풍차,제주카페 신창풍차해안도로 카페파람 한라봉스무디...,#제주여행 #제주카페 #한경카페\n#제주카페추천 #한경카페추천 \n#신창풍차해안도로...,https://blog.naver.com/dengvely?Redirect=Log&l...,댕블리
